In [1]:
import pandas as pd
import keras

Using TensorFlow backend.


In [34]:
df = pd.read_csv('data.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.describe()

,d1,d2,d3,d4,d5,d6
count,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000
mean,27.516883,21.197193,21.383774,-inf,15.252616,26.051990
std,2.464038,2.917632,1.437963,NaN,2.838004,0.954564
min,-4.945500,5.547100,7.527200,-inf,2.218000,12.863000
25%,27.489000,19.963000,20.619000,9.852700,13.290000,25.594000
50%,28.026000,21.823000,21.512000,11.761000,15.140000,26.067000
75%,28.506000,23.087000,22.338000,13.080000,17.269000,26.563000
max,31.817000,27.959000,25.594000,21.161000,27.682000,28.920000


In [3]:
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import LSTM


## Build dataset
### dataRNN is 1*6
### dataCNN is 2*3

In [4]:
import numpy as np
dataset = []
for i in df.index:
    dataset.append(df.loc[i])
    
dataset = np.array(dataset)

In [5]:
dataRNN = dataset
dataRNN[0:3]

array([[ 30.521,  20.997,  23.296,  13.554,  23.684,  17.52 ],
       [ 29.793,  22.064,  22.486,  13.552,  23.43 ,  17.853],
       [ 30.487,  23.513,  24.087,  12.388,  22.616,  20.043]])

In [6]:
dataCNN = []
for i in dataRNN:
    dataCNN.append(np.reshape(i, (2,3)))
dataCNN = np.array(dataCNN)

In [7]:
dataCNN[0:3]

array([[[ 30.521,  20.997,  23.296],
        [ 13.554,  23.684,  17.52 ]],

       [[ 29.793,  22.064,  22.486],
        [ 13.552,  23.43 ,  17.853]],

       [[ 30.487,  23.513,  24.087],
        [ 12.388,  22.616,  20.043]]])

## RNN model

### Split into train and test set

In [8]:
train_size = int(len(dataRNN) * 0.67)
test_size = len(dataRNN) - train_size
print(train_size,test_size)
train = dataRNN[0:train_size,:]
test = dataRNN[train_size:len(dataRNN),:]

2219 1094


### convert an array of values into a data matrix

In [10]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        dataX.append(a)
        dataY.append(dataset[i + look_back, :])
    return np.array(dataX), np.array(dataY)

In [11]:
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [12]:
print(train.shape)
print(trainX.shape)
print(trainY.shape)

(2219, 6)
(2208, 10, 6)
(2208, 6)


### Reshape input to be [samples, time steps, features]

In [13]:
print(trainX[0].shape)
print(trainY[0].shape)
# trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

(10, 6)
(6,)


In [16]:
model = Sequential()
model.add(LSTM(24, input_shape=(10, 6)))
model.add(Dense(6))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 24)                2976      
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 150       
Total params: 3,126
Trainable params: 3,126
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)
score = model.evaluate(testX, testY)

Epoch 1/10
65s - loss: 103.1503
Epoch 2/10
59s - loss: 6.5164
Epoch 3/10
58s - loss: 4.6297
Epoch 4/10
59s - loss: 4.6101
Epoch 5/10
60s - loss: 4.5191
Epoch 6/10
60s - loss: 4.4464
Epoch 7/10
59s - loss: 4.3407
Epoch 8/10
64s - loss: 4.2112
Epoch 9/10
59s - loss: 4.1041
Epoch 10/10
59s - loss: 4.0061
 992/1083 [==========================>...] - ETA: 0s

In [21]:
data = testX[100]
data

array([[ 28.301 ,  22.335 ,  21.419 ,  -4.7714,  19.563 ,  26.965 ],
       [ 28.599 ,  22.647 ,  20.071 ,   1.2492,  20.452 ,  27.734 ],
       [ 27.684 ,  21.878 ,  20.898 ,   5.2285,  20.525 ,  26.386 ],
       [ 27.876 ,  22.374 ,  20.452 ,   5.2279,  19.378 ,  26.763 ],
       [ 28.08  ,  22.104 ,  20.452 ,   7.4583,  19.207 ,  27.648 ],
       [ 27.621 ,  22.077 ,  20.085 ,   7.7814,  18.345 ,  26.629 ],
       [ 27.292 ,  22.697 ,  21.672 ,   8.2389,  17.609 ,  26.41  ],
       [ 27.575 ,  21.332 ,  19.461 ,   9.3784,  16.368 ,  26.141 ],
       [ 27.812 ,  21.22  ,  20.543 ,   9.3777,  16.355 ,  26.317 ],
       [ 27.489 ,  21.299 ,  21.355 ,   9.8526,  20.449 ,  27.075 ]])

In [23]:
data = np.reshape(data, (1,10,6))
data

array([[[ 28.301 ,  22.335 ,  21.419 ,  -4.7714,  19.563 ,  26.965 ],
        [ 28.599 ,  22.647 ,  20.071 ,   1.2492,  20.452 ,  27.734 ],
        [ 27.684 ,  21.878 ,  20.898 ,   5.2285,  20.525 ,  26.386 ],
        [ 27.876 ,  22.374 ,  20.452 ,   5.2279,  19.378 ,  26.763 ],
        [ 28.08  ,  22.104 ,  20.452 ,   7.4583,  19.207 ,  27.648 ],
        [ 27.621 ,  22.077 ,  20.085 ,   7.7814,  18.345 ,  26.629 ],
        [ 27.292 ,  22.697 ,  21.672 ,   8.2389,  17.609 ,  26.41  ],
        [ 27.575 ,  21.332 ,  19.461 ,   9.3784,  16.368 ,  26.141 ],
        [ 27.812 ,  21.22  ,  20.543 ,   9.3777,  16.355 ,  26.317 ],
        [ 27.489 ,  21.299 ,  21.355 ,   9.8526,  20.449 ,  27.075 ]]])

In [24]:
pre = model.predict(data)

In [25]:
pre

array([[ 25.75363159,  20.51769447,  21.10876083,  10.53987217,
         14.42281723,  25.43008995]], dtype=float32)

In [26]:
testY[100]

array([ 29.643 ,  21.656 ,  20.91  ,   7.5331,  13.357 ,  25.908 ])

### data1

In [30]:
df = pd.read_csv('data1.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()
df.describe()

,d1,d2,d3,d4,d5,d6
count,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000
mean,27.011211,24.296531,22.851550,27.929205,28.015615,19.454348
std,1.613423,2.583492,1.363558,0.559885,1.018381,1.891391
min,7.239700,9.289700,13.861000,22.840000,14.366000,8.238600
25%,26.687000,23.500000,22.059000,27.586000,27.486000,18.408000
50%,27.217000,25.273000,22.907000,27.914000,28.133000,19.606000
75%,27.674000,25.995000,23.719000,28.291000,28.672000,20.705000
max,32.641000,28.196000,27.483000,30.975000,31.435000,27.735000


In [28]:
import numpy as np
dataRNN1 = []
for i in df.index:
    dataRNN1.append(df.loc[i])
    
dataRNN = np.array(dataRNN)
train_size = int(len(dataRNN) * 0.67)
test_size = len(dataRNN) - train_size
print(train_size,test_size)
train = dataRNN[0:train_size,:]
test = dataRNN[train_size:len(dataRNN),:]
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

2219 1094


In [29]:
model = Sequential()
model.add(LSTM(24, input_shape=(10, 6)))
model.add(Dense(6))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)
score = model.evaluate(testX, testY)

Epoch 1/10
65s - loss: 151.6627
Epoch 2/10
63s - loss: 6.1711
Epoch 3/10
64s - loss: 4.6019
Epoch 4/10
63s - loss: 4.5017
Epoch 5/10
68s - loss: 4.4430
Epoch 6/10
71s - loss: 4.3447
Epoch 7/10
69s - loss: 4.8007
Epoch 8/10
70s - loss: 4.1961
Epoch 9/10
71s - loss: 4.1103
Epoch 10/10
72s - loss: 4.0146
 928/1083 [========================>.....] - ETA: 0s

### data 2

In [31]:
df = pd.read_csv('data2.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()
df.describe()

,d1,d2,d3,d4,d5,d6
count,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000
mean,24.811039,20.328523,23.739113,14.021725,-inf,17.804222
std,2.325009,2.633894,1.161522,2.234414,NaN,1.907775
min,-2.037200,3.160000,15.270000,-1.761200,-inf,7.531700
25%,24.546000,19.478000,23.102000,12.389000,21.419000,16.842000
50%,25.304000,21.031000,23.879000,13.921000,22.218000,17.609000
75%,25.856000,21.967000,24.512000,15.229000,23.186000,18.920000
max,30.672000,26.711000,26.440000,24.065000,26.960000,27.483000


In [32]:
import numpy as np
dataRNN1 = []
for i in df.index:
    dataRNN1.append(df.loc[i])
    
dataRNN = np.array(dataRNN)
train_size = int(len(dataRNN) * 0.67)
test_size = len(dataRNN) - train_size
print(train_size,test_size)
train = dataRNN[0:train_size,:]
test = dataRNN[train_size:len(dataRNN),:]
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

2219 1094


In [35]:
model = Sequential()
model.add(LSTM(24, input_shape=(10, 6)))
model.add(Dense(6))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)
score = model.evaluate(testX, testY)

Epoch 1/10
63s - loss: 132.0817
Epoch 2/10
59s - loss: 5.9731
Epoch 3/10
58s - loss: 4.6234
Epoch 4/10
58s - loss: 4.4872
Epoch 5/10
59s - loss: 4.3931
Epoch 6/10
59s - loss: 4.2646
Epoch 7/10
60s - loss: 4.1595
Epoch 8/10
61s - loss: 4.0470
Epoch 9/10
64s - loss: 3.9255
Epoch 10/10
60s - loss: 3.8406
 992/1083 [==========================>...] - ETA: 0s 

### data 3

In [36]:
df = pd.read_csv('data3.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()
df.describe()

,d1,d2,d3,d4,d5,d6
count,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000
mean,25.265554,21.243057,21.908483,25.807361,22.638101,27.448187
std,3.068670,2.682293,1.062800,0.783282,2.253364,1.109782
min,2.999300,6.165600,15.271000,17.533000,4.259100,-1.761400
25%,24.045000,20.393000,21.303000,25.399000,21.092000,26.918000
50%,26.186000,22.075000,21.912000,25.858000,22.847000,27.515000
75%,27.231000,23.075000,22.535000,26.301000,24.414000,28.070000
max,31.538000,25.611000,26.695000,28.301000,28.850000,30.238000


In [37]:
import numpy as np
dataRNN1 = []
for i in df.index:
    dataRNN1.append(df.loc[i])
    
dataRNN = np.array(dataRNN)
train_size = int(len(dataRNN) * 0.67)
test_size = len(dataRNN) - train_size
print(train_size,test_size)
train = dataRNN[0:train_size,:]
test = dataRNN[train_size:len(dataRNN),:]
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

2219 1094


In [38]:
model = Sequential()
model.add(LSTM(24, input_shape=(10, 6)))
model.add(Dense(6))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)
score = model.evaluate(testX, testY)

Epoch 1/10
67s - loss: 177.5793
Epoch 2/10
65s - loss: 8.4374
Epoch 3/10
65s - loss: 4.6064
Epoch 4/10
65s - loss: 4.4499
Epoch 5/10
65s - loss: 4.2364
Epoch 6/10
65s - loss: 4.0507
Epoch 7/10
65s - loss: 3.9310
Epoch 8/10
65s - loss: 3.8342
Epoch 9/10
65s - loss: 3.7978
Epoch 10/10
66s - loss: 3.7918
1056/1083 [============================>.] - ETA: 0s 